In [59]:
from langchain_openai import ChatOpenAI
from langchain_community.llms import Bedrock
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from utils import get_files_and_contents
from getpass import getpass


# ACCESS_TOKEN = getpass()

In [48]:
files_in_repo = get_files_and_contents("llm.sh")

In [49]:
sample_user_query = r""" 
(base) C:\Users\off99\Documents\Code\>llm list files in current dir; windows
/ Querying GPT-3200
───────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       │ File: temp.sh
───────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1   │
   2   │ dir
   3   │ ```
───────┴────────────────────────────────────────────────────────────────────────────────────────────────────────────────
>> Do you want to run this program? [Y/n] y

Running...


(base) C:\Users\off99\Documents\Code\>
Notice that there is no output. Is this supposed to work on Windows also?
Also it might be great if the script detects which OS or shell I'm using and try to use the appropriate command e.g. dir instead of ls because I don't want to be adding windows after every prompt.


"""

In [50]:
llm = ChatOpenAI(model_name="gpt-4-0125-preview")
# llm = Bedrock(model_id="anthropic.claude-v2:1")

In [51]:
fixing_prompt = PromptTemplate.from_template(
    """You are an expert at generating Git Diffs. Given a user query defined under <user_query><user_query/>,
    you can identify the files that you need to update to solve the problem/issue that the user is having. 
    These files are given as context to you in between <repository></repository>.

    <repository>
    {repository}
    </repository>

    Given these files, do the following step by step:
    1. Identify what files need to be changed to solve user's query
    2. Identify what changes need to be made in the files
    3. generate a git diff showing all of the changes that you have made
    4. lastly, check your work to make sure these are the changes you would like to make to solve the user's query

    Make sure to only return the diff and no other text.
    
    <user_query>
    {user_query}
    </user_query>
    """
)

In [80]:
reflection_prompt = PromptTemplate.from_template(
    """ 
    The user wants to update the functionality of the code within the given repository and you have generated the git diff
    for the code changes located in <diff></diff>. The issue that the user was facing is located below in <user_query></user_query>. 
    Additionally, the repository that the user is referencing is loccated in <repository></repository>. The repository section contains
    all of the files. You can use it to verify if it is updating the correct code section and correct file


    <user_query>
    {user_query}
    </user_query>

    <diff>
    {diff}
    </diff>

    <repository>
    {repository}
    </repository>

    Now, are you sure these are the changes that you want to make in the code? Are you positive that the code changes within the git diff will solve the user's query? If you want to correct
    your answer, then generate a new git diff and then return it!

    Only return the git diff and no other text.
    """
)

In [89]:
generate_diff_chain = (
    {
        "diff": (
            RunnablePassthrough.assign(
                repository=lambda x: files_in_repo
            )
            | fixing_prompt
            | llm
            | StrOutputParser() 
        ),
        "repository": lambda x: files_in_repo,
        "user_query": RunnablePassthrough(),
    }
)

reflection_chain = reflection_prompt | llm | StrOutputParser()

final_chain = generate_diff_chain | reflection_chain


In [91]:
response = final_chain.invoke({"user_query": sample_user_query})

In [93]:
print(response)

```diff
diff --git a/llm.sh/src/main.py b/llm.sh/src/main.py
index d7e5263..b4a1f87 100644
--- a/llm.sh/src/main.py
+++ b/llm.sh/src/main.py
@@ -1,5 +1,6 @@
 import os
 import sys
+import platform
 from colorama import Fore
 from halo import Halo
 import requests
@@ -38,6 +39,11 @@ def model_query(prompt: str) -> str:
     return response.json()['text']
 
 
+def get_os_command(command: str) -> str:
+    if platform.system() == "Windows":
+        return command.replace("ls", "dir")
+    return command
+
 def process():
     prompt = ' '.join(sys.argv[1:])
     result = model_query(prompt)
@@ -45,6 +51,7 @@ def process():
     response = input(Fore.RED + '>> Do you want to run this program? [Y/n] ')
     if response == '' or response.lower() == 'y':
         print(Fore.LIGHTBLACK_EX + '\nRunning...\n')
+        result = get_os_command(result)
         run_bash_file_from_string(result)
     else:
         print(Fore.LIGHTBLACK_EX + 'Aborted.')
```


```diff
diff --git a/llm.sh/src/main.py b/llm.sh/src/main.py
index d7e5263..b4a1f87 100644
--- a/llm.sh/src/main.py
+++ b/llm.sh/src/main.py
@@ -1,5 +1,6 @@
 import os
 import sys
+import platform
 from colorama import Fore
 from halo import Halo
 import requests
@@ -38,6 +39,11 @@ def model_query(prompt: str) -> str:
     return response.json()['text']
 
 
+def get_os_command(command: str) -> str:
+    if platform.system() == "Windows":
+        return command.replace("ls", "dir")
+    return command
+
 def process():
     prompt = ' '.join(sys.argv[1:])
     result = model_query(prompt)
@@ -45,6 +51,7 @@ def process():
     response = input(Fore.RED + '>> Do you want to run this program? [Y/n] ')
     if response == '' or response.lower() == 'y':
         print(Fore.LIGHTBLACK_EX + '\nRunning...\n')
+        result = get_os_command(result)
         run_bash_file_from_string(result)
     else:
         print(Fore.LIGHTBLACK_EX + 'Aborted.')
```


In [ ]:
# Github Access Token currently not working right now

GITHUB_URL = "octokit/octokit.rb"

from langchain_community.document_loaders import GithubFileLoader

loader = GithubFileLoader(
    repo=GITHUB_URL,  # the repo name
    access_token=ACCESS_TOKEN,
    github_api_url="https://api.github.com",
)

documents = loader.load()